# Setup

In [ ]:
import os
import datetime
import pandas as pd
import requests

# Read data from external source

In [ ]:
# Attempt to read current product stock data
products_url = "https://dummyjson.com/products"
products_response = requests.get(products_url)

# No valid response or json-data returned
# Fail fast, break it all (nothing else works without this, no chance to recover)
if not (products_response and 
        products_response.status_code == 200 and 
        "application/json" in products_response.headers.get("Content-Type", "")):
    raise ValueError("No json content to work with")

# Get json content from response (we know it's good by now)
products_json = products_response.json()

# Process data 

In [ ]:
# Intial cleaning of data - we don't need the metadata (limit, total etc)
# We only want the products
products_json_cleaned = products_json["products"]

# Iterate list of products and remove unnecessary data (images/thumbnail)
unnecessary_data = ["images", "thumbnail"]
for product in products_json_cleaned:
    for key in unnecessary_data:
        product.pop(key, "")

# Create a dataframe to work with, simplify processing
products_df = pd.DataFrame(products_json_cleaned)

# Filter data based on criterea
# If an item is on sale (discount > 10%) and it has low stock count (stock < 50) --> must order more!
products_to_order_df = products_df[(products_df["discountPercentage"] > 10) & (products_df["stock"] < 50)]
# Sort by stock, lowest up top
products_to_order_df.sort_values(by="stock")

# Produce output

In [ ]:
# Create excel file with items to order

# Get today's date for the folder name and time for file name
today = datetime.datetime.now()
date_today = today.strftime("%x").replace("/","-")

# Try to find a folder for today, if there isn't one then create it
folder_name = f"Orders-{date_today}"
if os.path.exists(folder_name) and os.path.isdir(folder_name):
    pass # it exists! do nothing :)
else:
    os.mkdir(folder_name) # create the folder

# Write our file inside our folder
products_to_order_df.to_excel(f"{folder_name}/Orders.xlsx", index=None)